In [1]:
# !pip install numpy==1.23.5
# !pip install protobuf==3.19.*
#!pip install protobuf==3.19.0
#!pip install pyyaml h5py
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Recall, Precision 

import os
import pickle
import pandas as pd
import numpy as np
from model_team14 import *

In [2]:


# ## select imputed & transformed data
# X=pd.read_csv('../data/X_data_tr.csv', index_col='date', parse_dates=True)
# y=pd.read_csv('../data/y_data_tr.csv', index_col='date', parse_dates=True)
# threshold=0.2
# criteria=None

# metadata=pd.read_csv('../data/full_info.csv')
# df_feature=select_features(metadata, X, threshold, criteria=criteria)

def get_data(X, y, y_type, test_year, features):
    X_train=X[features][:-(test_year*12)]
    y_train=y[y_type][:-(test_year*12)]           
    X_test=X[features][-(test_year*12):]   
    y_test=y[y_type][-(test_year*12):]

    return X_train, y_train, X_test, y_test

In [3]:
## time_step means the length(n) of explanatory variables(t-(n-1) ~ t) to use for classifying y in certain time(t) period.

def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [4]:
def grid_search(path, trainX, trainy, testX, testy, y_type, nl=[50,100], epochs=[100,300], batch_sizes=[10], time_steps=[18,24],
                n_layer=[50,100]):
    results = []
    best_auc = 0
    best_params = None
    nc=len(trainy.unique())


    for time_step in time_steps:
        X_train, y_train = create_dataset(trainX, trainy, time_step)
        X_test, y_test = create_dataset(testX, testy, time_step)
        y_train = to_categorical(y_train, num_classes=nc)
        y_test = to_categorical(y_test, num_classes=nc)
        
        loss_type = 'categorical_crossentropy'   ##if y_type == 'y_agg' else 'binary_crossentropy'

    
        for num_epochs in epochs:
            for batch_size in batch_sizes:
                for num_layer in nl:
                    # create the model
                    model = Sequential()
                    model.add(LSTM(num_layer, input_shape=(X_train.shape[1], X_train.shape[2])))  ##input_shape=(X_train.shape[1], X_train.shape[2])
                    # model.add(Dropout(0.2))
                    #model.add(Dense(2))
                    model.add(Dense(nc, activation='softmax'))

                    model.compile(loss=loss_type, optimizer='adam', metrics=['AUC','Accuracy',
                                                                             'Recall', 
                                                                             'Precision'])
#                    model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size)
                    model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size)

#                    print(model.summary())

                    # Final evaluation of the model
                    scores = model.evaluate(X_test, y_test, verbose=0)
                    #print(scores)
                    auc = scores[1]
                    acc = scores[2]
                    recall = scores[3]
                    precision = scores[4]
                    #print("AUC: %.2f%%" % (auc))
                    
                    model_name='clf_lstm_{}_{}_b{}_ep{}_h{}'.format(y_type, time_step, batch_size, num_epochs, num_layer)
                    model.save(path+'/{}.h5'.format(model_name))
                    current_result = {'model':model_name,
                                      'precision': precision,  #"%.2f%%" % (precision)
                                      'recall': recall,
                                      'accuracy': acc,
                                      'auc': auc
                                     }
                    results.append(current_result)

                    if auc > best_auc:
                        best_auc = auc
                        best_params = current_result

#    print("Best AUC: ", best_params)
    return results


In [5]:
y_types = ['y_agg','y_oecd']
thresholds=[0.2,0.3,0.4,0.5]
criteria=None
test_year=5

## select imputed & transformed data
X=pd.read_csv('../data/X_data.csv', index_col='date', parse_dates=True) ##'../data/X_data_tr.csv'
y=pd.read_csv('../data/y_data.csv', index_col='date', parse_dates=True) ##'../data/X_data_tr.csv'
metadata=pd.read_csv('../data/full_info.csv')

final_results=pd.DataFrame()

for i, threshold in enumerate(thresholds):
    df_feature=select_features(metadata, X, threshold, criteria=criteria)
    selected_features=list(df_feature[df_feature.select==1]['variable'])
    print(threshold, selected_features, len(selected_features))
    
    path='../result/ft_{}_{}_ls_ntr'.format(str(criteria), str(threshold))

    if not os.path.exists(path):
        os.mkdir(path)
        
    for y_type in y_types:
        X_train, y_train, X_test, y_test = get_data(X, y, y_type, test_year, selected_features)
        
        results=grid_search(path, X_train, y_train, X_test, y_test, y_type, nl=[50,100],  epochs=[300,500], batch_sizes=[5], time_steps=[18,24])
#        results=grid_search(path, X_train, y_train, X_test, y_test, y_type, nl=[5],  epochs=[10], batch_sizes=[5], time_steps=[1,2])

        df_result=pd.DataFrame(results)
        df_result['ft_criteria']=criteria
        df_result['ft_threshold']=threshold
        
        final_results=pd.concat([final_results, df_result])

# with open('../result/df_lstm_creport.pkl', 'wb') as f:
#     pickle.dump(final_results, f)

    # print(grid_search(X_train, y_train, X_test, y_test, y_type, nl=[50,100],  epochs=[300,500], batch_sizes=[5], time_steps=[18,24]))
    ## ,  epochs=[100,300], batch_sizes=[5,10], time_steps=[18,24]



0.2 ['INDPRO', 'SRVPRD', 'HOUST', 'CMRMTSPLx', 'M2REAL', 'FEDFUNDS', '5Y3M', 'WPSFD49207', 'S&P 500', 'BSI', 'CSI', 'H8B1151NCBCMG'] 12
Epoch 1/300
102/102 [==============================] - 1s 3ms/step - loss: 0.9500 - auc: 0.7237 - Accuracy: 0.5373 - recall: 0.3333 - precision: 0.5667 
Epoch 2/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9286 - auc: 0.7261 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 3/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9305 - auc: 0.7212 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 4/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9294 - auc: 0.7189 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 5/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9275 - auc: 0.7285 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 6/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9295 - auc: 0.7123 - Accur

102/102 [==============================] - 0s 3ms/step - loss: 0.9288 - auc: 0.7233 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 107/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9292 - auc: 0.7183 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 108/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9286 - auc: 0.7167 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 109/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9289 - auc: 0.7221 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 110/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9285 - auc: 0.7196 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 111/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9287 - auc: 0.7203 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 112/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9281 - auc:

102/102 [==============================] - 0s 4ms/step - loss: 0.9290 - auc: 0.7273 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 213/300
102/102 [==============================] - 0s 4ms/step - loss: 0.9295 - auc: 0.7197 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 214/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9282 - auc: 0.7242 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 215/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9292 - auc: 0.7131 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 216/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9290 - auc: 0.7229 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 217/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9289 - auc: 0.7105 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 218/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9299 - auc:

102/102 [==============================] - 0s 4ms/step - loss: 0.9367 - auc: 0.7174 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 19/300
102/102 [==============================] - 0s 4ms/step - loss: 0.9317 - auc: 0.7169 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 20/300
102/102 [==============================] - 0s 4ms/step - loss: 0.9293 - auc: 0.7239 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 21/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9309 - auc: 0.7168 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 22/300
102/102 [==============================] - 0s 4ms/step - loss: 0.9308 - auc: 0.7241 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 23/300
102/102 [==============================] - 0s 3ms/step - loss: 0.9329 - auc: 0.7183 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 24/300
102/102 [==============================] - 0s 4ms/step - loss: 0.9307 - auc: 0.712

102/102 [==============================] - 0s 5ms/step - loss: 0.9298 - auc: 0.7191 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 125/300
102/102 [==============================] - 0s 4ms/step - loss: 0.9327 - auc: 0.7220 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 126/300
102/102 [==============================] - 0s 5ms/step - loss: 0.9313 - auc: 0.7207 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 127/300
102/102 [==============================] - 0s 4ms/step - loss: 0.9319 - auc: 0.7133 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 128/300
102/102 [==============================] - 0s 4ms/step - loss: 0.9304 - auc: 0.7243 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 129/300
102/102 [==============================] - 0s 5ms/step - loss: 0.9304 - auc: 0.7187 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 130/300
102/102 [==============================] - 0s 5ms/step - loss: 0.9323 - auc:

...

101/101 [==============================] - 1s 14ms/step - loss: 0.6888 - auc: 0.5459 - Accuracy: 0.5655 - recall: 0.5655 - precision: 0.5655
Epoch 319/500
101/101 [==============================] - 2s 15ms/step - loss: 0.6870 - auc: 0.5570 - Accuracy: 0.5655 - recall: 0.5655 - precision: 0.5655
Epoch 320/500
101/101 [==============================] - 2s 15ms/step - loss: 0.6882 - auc: 0.5496 - Accuracy: 0.5655 - recall: 0.5655 - precision: 0.5655
Epoch 321/500
101/101 [==============================] - 1s 13ms/step - loss: 0.6878 - auc: 0.5510 - Accuracy: 0.5655 - recall: 0.5655 - precision: 0.5655
Epoch 322/500
101/101 [==============================] - 1s 13ms/step - loss: 0.6859 - auc: 0.5549 - Accuracy: 0.5655 - recall: 0.5655 - precision: 0.5655
Epoch 323/500
101/101 [==============================] - 1s 14ms/step - loss: 0.6859 - auc: 0.5458 - Accuracy: 0.5655 - recall: 0.5655 - precision: 0.5655
Epoch 324/500
101/101 [==============================] - 1s 15ms/step - loss: 0.6879

101/101 [==============================] - 2s 17ms/step - loss: 0.6854 - auc: 0.5713 - Accuracy: 0.5655 - recall: 0.5655 - precision: 0.5655
Epoch 423/500
101/101 [==============================] - 2s 19ms/step - loss: 0.6865 - auc: 0.5606 - Accuracy: 0.5655 - recall: 0.5655 - precision: 0.5655
Epoch 424/500
101/101 [==============================] - 1s 15ms/step - loss: 0.6859 - auc: 0.5642 - Accuracy: 0.5655 - recall: 0.5655 - precision: 0.5655
Epoch 425/500
101/101 [==============================] - 2s 18ms/step - loss: 0.6861 - auc: 0.5448 - Accuracy: 0.5655 - recall: 0.5655 - precision: 0.5655
Epoch 426/500
101/101 [==============================] - 2s 16ms/step - loss: 0.6854 - auc: 0.5646 - Accuracy: 0.5655 - recall: 0.5655 - precision: 0.5655
Epoch 427/500
101/101 [==============================] - 2s 15ms/step - loss: 0.6869 - auc: 0.5614 - Accuracy: 0.5655 - recall: 0.5655 - precision: 0.5655
Epoch 428/500
101/101 [==============================] - 2s 17ms/step - loss: 0.6874

102/102 [==============================] - 1s 7ms/step - loss: 0.9311 - auc: 0.7172 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 27/300
102/102 [==============================] - 1s 8ms/step - loss: 0.9294 - auc: 0.7202 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 28/300
102/102 [==============================] - 1s 8ms/step - loss: 0.9291 - auc: 0.7104 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 29/300
102/102 [==============================] - 1s 9ms/step - loss: 0.9296 - auc: 0.7112 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 30/300
102/102 [==============================] - 1s 10ms/step - loss: 0.9290 - auc: 0.7197 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 31/300
102/102 [==============================] - 1s 7ms/step - loss: 0.9296 - auc: 0.7238 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 32/300
102/102 [==============================] - 1s 10ms/step - loss: 0.9291 - auc: 0.7

102/102 [==============================] - 1s 11ms/step - loss: 0.9291 - auc: 0.7208 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 133/300
102/102 [==============================] - 1s 9ms/step - loss: 0.9281 - auc: 0.7273 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 134/300
102/102 [==============================] - 1s 9ms/step - loss: 0.9293 - auc: 0.7172 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 135/300
102/102 [==============================] - 1s 7ms/step - loss: 0.9287 - auc: 0.7221 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 136/300
102/102 [==============================] - 1s 8ms/step - loss: 0.9287 - auc: 0.7201 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 137/300
102/102 [==============================] - 1s 9ms/step - loss: 0.9293 - auc: 0.7205 - Accuracy: 0.5686 - recall: 0.5686 - precision: 0.5686
Epoch 138/300
102/102 [==============================] - 1s 8ms/step - loss: 0.9289 - auc

KeyboardInterrupt: 

In [ ]:
# with open('../result/df_lstm_creport.pkl', 'rb') as f:
#     df_lstm_report=pickle.load(f)

# df_lstm_report

In [ ]:
# with open('../result/df_lstm_creport.pkl', 'rb') as f:
#     final_results=pickle.load(f)

# clf_lstm_y_oecd_3_b5_ep2_h4

In [ ]:
# y_type = 'y_oecd'
# X_train, y_train, X_test, y_test = get_data(y_type, 5, selected_features)

# # print(grid_search(X_train, y_train, X_test, y_test, y_type, nl=[10,20],  epochs=[5,10], batch_sizes=[5], time_steps=[18]))
# ## ,  epochs=[100,300], batch_sizes=[5,10], time_steps=[18,24]

# result_dict_oecd=grid_search(X_train, y_train, X_test, y_test, y_type, nl=[10],  epochs=[10], batch_sizes=[5], time_steps=[18])

In [ ]:
# pd.DataFrame(result_dict_agg)

In [ ]:
# pd.DataFrame(result_dict_oecd)

In [ ]:

# model2=result_dict[1]['model']

# print(model, model2)

# dir(model)
# model.summary()

In [ ]:
# !pip uninstall shap
# !pip install shap

In [ ]:
# import pickle
# with open('../result/result_dict_agg.pkl','wb') as f:
#     pickle.dump(result_dict_agg, f)

# with open('../result/result_dict_oecd.pkl','wb') as f:
#     pickle.dump(result_dict_oecd, f)


In [ ]:
# !pip uninstall numba
# #!pip install --upgrade numba

In [ ]:
#import shap
# import model_team14 
# from model_team14 import *  ##select_features, plot_pca, DTW

# import os
# import pandas as pd
# import numpy as np
# import pickle
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import label_binarize
# from sklearn.metrics import confusion_matrix,accuracy_score, recall_score, precision_score, f1_score,classification_report 
# from sklearn.metrics import roc_curve, auc

# import seaborn as sns


In [ ]:
# model.predict(np.array(X_test_shap))
# model.fit(np.array(X_test_shap), np.array(y_train_shap))
# model.summary()
# X_train_shap.shape, X_train.shape

In [ ]:
# ##https://stackoverflow.com/questions/66814523/shap-deepexplainer-with-tensorflow-2-4-error?noredirect=1
# ## this might be helpful your debugging!!!

# import shap
# import pickle
# tf.compat.v1.disable_v2_behavior()  ## input type err solved

# # import tensorflow.compat.v1.keras.backend as K
# # import tensorflow as tf
# # tf.compat.v1.disable_eager_execution()
# # tf version is 2.3.1
# # kerase version is 2.4.0
# # Shap version is 0.36

# # with open ('../result/result_dict_oecd.pkl','rb') as f:
# #     result_dict_oecd=pickle.load(f)

# model=result_dict_oecd[0]['model']

# y_type = 'y_oecd'
# X_train, y_train, X_test, y_test = get_data(y_type, 5, selected_features)


# X_train_shap, y_train_shap = create_dataset(X_train, y_train, 18)
# X_test_shap, y_test_shap = create_dataset(X_test, y_test, 18)

# y_train_shap = to_categorical(y_train_shap, num_classes=2)
# y_test_shap = to_categorical(y_test_shap, num_classes=2)

# # X_train_shap=tf.convert_to_tensor(X_train_shap)
# # X_test_shap=tf.convert_to_tensor(X_test_shap)

# X_train_shap=np.array(X_train_shap)
# y_train_shap=np.array(y_train_shap)
# X_test_shap=np.array(X_test_shap)
# y_test_shap=np.array(y_test_shap)

# #type(X_train_shap)
# #X_train

# model.fit(X_train_shap, y_train_shap, batch_size=5, epochs=5)


# X_train_shap2=tf.convert_to_tensor(X_train_shap)
# X_test_shap2=tf.convert_to_tensor(X_test_shap)

# #explainer=shap.DeepExplainer(model, X_train_shap)   #works!

# explainer=shap.DeepExplainer(model, X_train_shap)   #works!

# values=explainer.shap_values(np.array(X_test_shap))



In [ ]:
# shap.initjs()
# shap.force_plot(explainer.expected_value[0], values[0][35], selected_features)

In [ ]:
# shap.initjs()
# shap.force_plot(explainer.expected_value[1], values[0][35], selected_features)

In [ ]:
# dir(explainer)
# explainer.expected_value

In [ ]:
# ##https://stackoverflow.com/questions/66814523/shap-deepexplainer-with-tensorflow-2-4-error?noredirect=1
# ## this might be helpful your debugging!!!

# import shap
# import pickle
# tf.compat.v1.disable_v2_behavior()  ## input type err solved

# # import tensorflow.compat.v1.keras.backend as K
# # import tensorflow as tf
# # tf.compat.v1.disable_eager_execution()
# # tf version is 2.3.1
# # kerase version is 2.4.0
# # Shap version is 0.36

# # with open ('../result/result_dict_oecd.pkl','rb') as f:
# #     result_dict_oecd=pickle.load(f)


# y_type = 'y_oecd'
# X_train, y_train, X_test, y_test = get_data(y_type, 5, selected_features)


# X_train_shap, y_train_shap = create_dataset(X_train, y_train, 18)
# X_test_shap, y_test_shap = create_dataset(X_test, y_test, 18)

# y_train_shap = to_categorical(y_train_shap, num_classes=2)
# y_test_shap = to_categorical(y_test_shap, num_classes=2)

# # X_train_shap=tf.convert_to_tensor(X_train_shap)
# # X_test_shap=tf.convert_to_tensor(X_test_shap)

# X_train_shap=np.array(X_train_shap)
# y_train_shap=np.array(y_train_shap)
# X_test_shap=np.array(X_test_shap)
# y_test_shap=np.array(y_test_shap)

# #type(X_train_shap)
# #X_train

# new_model.fit(X_train_shap, y_train_shap, batch_size=5, epochs=5)


# X_train_shap2=tf.convert_to_tensor(X_train_shap)
# X_test_shap2=tf.convert_to_tensor(X_test_shap)

# #explainer=shap.DeepExplainer(model, X_train_shap)   #works!

# explainer=shap.DeepExplainer(new_model, X_train_shap)   #works!

# values=explainer.shap_values(np.array(X_test_shap))



In [ ]:
# explainer.expected_value

In [ ]:
# # values[0].shape
# # shap.summary_plot(values[0], X_test_shap)
# # values[0]
# shap.initjs()
# shap.force_plot(explainer.expected_value[0], values[0][35], selected_features)